<a href="https://colab.research.google.com/github/nisaral/Liat.ai_Assignment/blob/main/liat_ai_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [4]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import pickle
from scipy.optimize import linear_sum_assignment
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
from collections import defaultdict, deque
import matplotlib.pyplot as plt

class PlayerFeatureExtractor:
    """Extract features for player re-identification"""

    def __init__(self):
        self.color_bins = 32

    def extract_color_histogram(self, image, mask=None):
        """Extract color histogram features"""
        if image.size == 0:
            return np.zeros(self.color_bins * self.color_bins * self.color_bins)

        if mask is not None:
            image = cv2.bitwise_and(image, image, mask=mask)

        # Convert to HSV for better color representation
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Calculate histogram
        hist = cv2.calcHist([hsv], [0, 1, 2], mask,
                           [self.color_bins, self.color_bins, self.color_bins],
                           [0, 180, 0, 256, 0, 256])

        # Normalize histogram
        hist = cv2.normalize(hist, hist).flatten()
        return hist

    def extract_position_features(self, bbox, frame_shape):
        """Extract normalized position features"""
        x1, y1, x2, y2 = bbox[:4]  # Ensure only first four values are used
        h, w = frame_shape[:2]

        # Normalize coordinates
        center_x = ((x1 + x2) / 2) / w
        center_y = ((y1 + y2) / 2) / h
        bbox_w = (x2 - x1) / w
        bbox_h = (y2 - y1) / h

        return np.array([center_x, center_y, bbox_w, bbox_h])

    def extract_motion_features(self, current_pos, prev_pos, time_diff=1):
        """Extract motion features"""
        if prev_pos is None:
            return np.array([0, 0, 0])

        vel_x = (current_pos[0] - prev_pos[0]) / time_diff
        vel_y = (current_pos[1] - prev_pos[1]) / time_diff
        speed = np.sqrt(vel_x**2 + vel_y**2)

        return np.array([vel_x, vel_y, speed])

class CrossCameraPlayerMapper:
    """Main class for cross-camera player mapping"""

    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        self.feature_extractor = PlayerFeatureExtractor()
        self.broadcast_players = defaultdict(list)
        self.tacticam_players = defaultdict(list)
        self.player_mappings = {}

    def detect_players(self, frame):
        """Detect players in frame using YOLO"""
        results = self.model(frame)
        players = []

        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Include players, goalkeepers, and referees (adjust class IDs as needed)
                    if int(box.cls) in [0, 1, 2]:  # 0: player, 1: goalkeeper, 2: referee
                        xyxy = box.xyxy[0].cpu().numpy()
                        conf = box.conf[0].cpu().numpy()
                        if conf > 0.5:  # Confidence threshold
                            players.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), conf])

        return players

    def extract_player_features(self, frame, bbox):
        """Extract comprehensive features for a player"""
        x1, y1, x2, y2 = bbox[:4]
        if x1 >= x2 or y1 >= y2 or x1 < 0 or y1 < 0 or x2 > frame.shape[1] or y2 > frame.shape[0]:
            return None

        # Crop player region
        player_crop = frame[y1:y2, x1:x2]
        if player_crop.size == 0:
            return None

        # Create mask for player region
        mask = np.ones((y2-y1, x2-x1), dtype=np.uint8) * 255

        # Extract features
        color_hist = self.feature_extractor.extract_color_histogram(player_crop, mask)
        pos_features = self.feature_extractor.extract_position_features(bbox, frame.shape)

        # Combine features
        features = np.concatenate([color_hist, pos_features])
        return features

    def process_video(self, video_path, camera_type='broadcast'):
        """Process video and extract player features"""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")

        frame_count = 0
        player_tracks = defaultdict(list)
        print(f"Processing {camera_type} video...")

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Detect players
            players = self.detect_players(frame)

            # Extract features for each player
            for i, player_bbox in enumerate(players):
                features = self.extract_player_features(frame, player_bbox)
                if features is not None:
                    player_data = {
                        'frame': frame_count,
                        'bbox': player_bbox,
                        'features': features,
                        'timestamp': frame_count / 30.0  # Assuming 30 FPS
                    }

                    # Simple tracking based on proximity
                    assigned = False
                    for track_id in player_tracks:
                        if len(player_tracks[track_id]) > 0:
                            last_pos = player_tracks[track_id][-1]['bbox']
                            current_pos = player_bbox

                            dist = np.sqrt((current_pos[0] - last_pos[0])**2 +
                                         (current_pos[1] - last_pos[1])**2)

                            if dist < 100:  # Proximity threshold
                                player_tracks[track_id].append(player_data)
                                assigned = True
                                break

                    if not assigned:
                        new_id = len(player_tracks)
                        player_tracks[new_id].append(player_data)

            frame_count += 1
            if frame_count % 100 == 0:
                print(f"Processed {frame_count} frames...")

        cap.release()

        if camera_type == 'broadcast':
            self.broadcast_players = player_tracks
        else:
            self.tacticam_players = player_tracks

        return player_tracks

    def calculate_similarity(self, features1, features2):
        """Calculate similarity between two feature vectors"""
        color_features1 = features1[:-4]
        color_features2 = features2[:-4]
        pos_features1 = features1[-4:]
        pos_features2 = features2[-4:]

        # Calculate color similarity
        color_sim = cosine_similarity([color_features1], [color_features2])[0][0]

        # Calculate position similarity
        pos_dist = np.linalg.norm(pos_features1[:2] - pos_features2[:2])
        pos_sim = 1 / (1 + pos_dist)

        # Calculate size similarity
        size_sim = 1 - abs(pos_features1[2] * pos_features1[3] - pos_features2[2] * pos_features2[3])

        # Weighted combination
        total_sim = 0.6 * color_sim + 0.3 * pos_sim + 0.1 * size_sim
        return total_sim

    def map_players_across_cameras(self):
        """Map players between broadcast and tacticam videos"""
        print("Mapping players across cameras...")

        broadcast_features = []
        tacticam_features = []
        broadcast_ids = []
        tacticam_ids = []

        # Aggregate features for each player
        for player_id, tracks in self.broadcast_players.items():
            if len(tracks) > 5:
                features_list = [track['features'] for track in tracks]
                median_features = np.median(features_list, axis=0)
                broadcast_features.append(median_features)
                broadcast_ids.append(player_id)

        for player_id, tracks in self.tacticam_players.items():
            if len(tracks) > 5:
                features_list = [track['features'] for track in tracks]
                median_features = np.median(features_list, axis=0)
                tacticam_features.append(median_features)
                tacticam_ids.append(player_id)

        if not broadcast_features or not tacticam_features:
            print("Warning: Insufficient player tracks for mapping.")
            return {}

        # Calculate similarity matrix
        similarity_matrix = np.zeros((len(broadcast_features), len(tacticam_features)))
        for i, b_feat in enumerate(broadcast_features):
            for j, t_feat in enumerate(tacticam_features):
                similarity_matrix[i, j] = self.calculate_similarity(b_feat, t_feat)

        # Use Hungarian algorithm
        row_ind, col_ind = linear_sum_assignment(-similarity_matrix)

        # Create mappings
        mappings = {}
        for i, j in zip(row_ind, col_ind):
            if similarity_matrix[i, j] > 0.5:
                mappings[broadcast_ids[i]] = tacticam_ids[j]

        self.player_mappings = mappings
        return mappings

    def visualize_mappings(self, broadcast_path, tacticam_path, output_path):
        """Create visualization of player mappings"""
        cap_b = cv2.VideoCapture(broadcast_path)
        cap_t = cv2.VideoCapture(tacticam_path)

        ret_b, frame_b = cap_b.read()
        ret_t, frame_t = cap_t.read()

        if ret_b and ret_t:
            for b_id, t_id in self.player_mappings.items():
                if b_id in self.broadcast_players and t_id in self.tacticam_players:
                    b_track = self.broadcast_players[b_id][0]
                    t_track = self.tacticam_players[t_id][0]

                    b_bbox = b_track['bbox']
                    cv2.rectangle(frame_b, (b_bbox[0], b_bbox[1]), (b_bbox[2], b_bbox[3]), (0, 255, 0), 2)
                    cv2.putText(frame_b, f'P{b_id}', (b_bbox[0], b_bbox[1]-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    t_bbox = t_track['bbox']
                    cv2.rectangle(frame_t, (t_bbox[0], t_bbox[1]), (t_bbox[2], t_bbox[3]), (0, 255, 0), 2)
                    cv2.putText(frame_t, f'P{b_id}', (t_bbox[0], t_bbox[1]-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            combined = np.hstack([frame_b, frame_t])
            cv2.imwrite(output_path, combined)

        cap_b.release()
        cap_t.release()

    def save_results(self, output_dir):
        """Save mapping results"""
        os.makedirs(output_dir, exist_ok=True)

        with open(os.path.join(output_dir, 'player_mappings.json'), 'w') as f:
            json.dump(self.player_mappings, f, indent=2)

        with open(os.path.join(output_dir, 'broadcast_tracks.pkl'), 'wb') as f:
            pickle.dump(dict(self.broadcast_players), f)

        with open(os.path.join(output_dir, 'tacticam_tracks.pkl'), 'wb') as f:
            pickle.dump(dict(self.tacticam_players), f)

        print(f"Results saved to {output_dir}")

def main():
    # Configuration
    MODEL_PATH = "best.pt"  # Relative path
    BROADCAST_VIDEO = "broadcast.mp4"
    TACTICAM_VIDEO = "tacticam.mp4"
    OUTPUT_DIR = "results"

    # Initialize mapper
    mapper = CrossCameraPlayerMapper(MODEL_PATH)

    # Process videos
    print("Processing broadcast video...")
    mapper.process_video(BROADCAST_VIDEO, 'broadcast')

    print("Processing tacticam video...")
    mapper.process_video(TACTICAM_VIDEO, 'tacticam')

    # Map players
    mappings = mapper.map_players_across_cameras()

    print(f"Found {len(mappings)} player mappings:")
    for b_id, t_id in mappings.items():
        print(f"Broadcast Player {b_id} -> Tacticam Player {t_id}")

    # Save results
    mapper.save_results(OUTPUT_DIR)

    # Create visualization
    mapper.visualize_mappings(BROADCAST_VIDEO, TACTICAM_VIDEO,
                             os.path.join(OUTPUT_DIR, 'mapping_visualization.jpg'))

    print("Cross-camera player mapping completed!")

if __name__ == "__main__":
    main()

Processing broadcast video...
Processing broadcast video...

0: 384x640 3 players, 2297.7ms
Speed: 5.3ms preprocess, 2297.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 2299.7ms
Speed: 4.2ms preprocess, 2299.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 3328.7ms
Speed: 4.3ms preprocess, 3328.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 2332.3ms
Speed: 4.9ms preprocess, 2332.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 2303.5ms
Speed: 4.1ms preprocess, 2303.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 1 referee, 2289.5ms
Speed: 3.2ms preprocess, 2289.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalkeeper, 12 players, 1 referee, 2286.1ms
Speed: 3.9ms preprocess, 2286.1ms inference, 1.1ms postprocess per image at shape (1

In [6]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import pickle
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import linear_sum_assignment
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
from collections import defaultdict, defaultdict
import matplotlib.pyplot as plt

class PlayerReIDFeatures:
    """Feature extraction for player re-identification"""

    def __init__(self):
        self.color_bins = 8
        self.label_encoder = LabelEncoder()

    def extract_appearance_features(self, frame, bbox):
        """Extract appearance-based features"""
        x1, y1, x2, y2 = bbox[:4]  # Ensure only first four values are used
        if x1 >= x2 or y1 >= y2 or x1 < 0 or y1 < 0 or x2 > frame.shape[1] or y2 > frame.shape[0]:
            return None

        # Crop player region
        player_crop = frame[y1:y2, x1:x2]
        if player_crop.size == 0:
            return None

        # Resize for consistency
        player_crop = cv2.resize(player_crop, (64, 128))

        # Convert to HSV
        hsv = cv2.cvtColor(player_crop, cv2.COLOR_BGR2HSV)
        hist_h = cv2.calcHist([hsv], [0], None, [self.color_bins], [0, 180])
        hist_s = cv2.calcHist([hsv], [1], None, [self.color_bins], [0, 256])
        hist_v = cv2.calcHist([hsv], [2], None, [self.color_bins], [0, 256])

        # Normalize histograms
        hist_h = cv2.normalize(hist_h, hist_h).flatten()
        hist_s = cv2.normalize(hist_s, hist_s).flatten()
        hist_v = cv2.normalize(hist_v, hist_v).flatten()

        # Combine color features
        color_features = np.concatenate([hist_h, hist_s, hist_v])

        # Texture features
        gray = cv2.cvtColor(player_crop, cv2.COLOR_BGR2GRAY)
        texture_features = self.extract_texture_features(gray)

        # Combine all appearance features
        return np.concatenate([color_features, texture_features])

    def extract_texture_features(self, gray_image):
        """Extract simple texture features"""
        # Calculate gradients
        grad_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)

        # Gradient magnitude
        magnitude = np.sqrt(grad_x**2 + grad_y**2)

        # Texture statistics
        texture_stats = [
            np.mean(magnitude),
            np.std(magnitude),
            np.mean(gray_image),
            np.std(gray_image)
        ]

        return np.array(texture_stats)

    def extract_spatial_features(self, frame, bbox):
        """Extract spatial/positional features"""
        x1, y1, x2, y2 = bbox[:4]
        h, w = frame.shape[:2]

        center_x = ((x1 + x2) / 2) / w
        center_y = ((y1 + y2) / 2) / h
        bbox_w = (x2 - x1) / w
        bbox_h = (y2 - y1) / h
        aspect_ratio = bbox_w / bbox_h if bbox_h > 0 else 0.0
        return np.array([center_x, center_y, bbox_w, bbox_h, aspect_ratio])

class PlayerTracker:
    """Player tracking with re-identification capabilities"""

    def __init__(self, model_path, max_disappeared=30, max_distance=100):
        self.model = YOLO(model_path)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        self.feature_extractor = PlayerReIDFeatures()

        self.max_disappeared = max_disappeared
        self.max_distance = max_distance

        self.next_id = 0
        self.active_players = {}
        self.disappeared_players = {}
        self.player_features = {}
        self.tracking_history = defaultdict(list)

        self.feature_history_size = 10
        self.reid_threshold = 0.6

    def detect_players(self, frame):
        """Detect players in frame"""
        results = self.model(frame, imgsz=640)  # Optimize inference speed
        detections = []

        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Include players and referees (adjust class IDs as needed)
                    if int(box.cls) in [0, 2]:  # 0: player, 2: referee
                        xyxy = box.xyxy[0].cpu().numpy()
                        conf = box.conf[0].cpu().numpy()
                        if conf > 0.5:
                            detections.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), conf])

        return detections

    def calculate_distance(self, bbox1, bbox2):
        """Calculate distance between two bounding boxes"""
        center1 = [(bbox1[0] + bbox1[2]) / 2, (bbox1[1] + bbox1[3]) / 2]
        center2 = [(bbox2[0] + bbox2[2]) / 2, (bbox2[1] + bbox2[3]) / 2]

        return np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)

    def calculate_feature_similarity(self, features1, features2):
        """Calculate similarity between feature vectors"""
        if features1 is None or features2 is None:
            return 0.0

        app_feat1 = features1[:-5]
        app_feat2 = features2[:-5]
        spatial_feat1 = features1[-5:]
        spatial_feat2 = features2[-5:]

        app_sim = cosine_similarity([app_feat1], [app_feat2])[0][0]
        spatial_sim = 1.0 / (1.0 + np.linalg.norm(spatial_feat1 - spatial_feat2))

        total_sim = 0.8 * app_sim + 0.2 * spatial_sim
        return max(0.0, total_sim)

    def extract_player_features(self, frame, bbox):
        """Extract comprehensive features for a player"""
        app_features = self.feature_extractor.extract_appearance_features(frame, bbox)
        if app_features is None:
            return None

        spatial_features = self.feature_extractor.extract_spatial_features(frame, bbox)
        return np.concatenate([app_features, spatial_features])

    def update_player_features(self, player_id, features):
        """Update feature history for a player"""
        if player_id not in self.player_features:
            self.player_features[player_id] = deque(maxlen=self.feature_history_size)
        self.player_features[player_id].append(features)

    def get_average_features(self, player_id):
        """Get average features for a player"""
        if player_id not in self.player_features or len(self.player_features[player_id]) == 0:
            return None
        return np.mean(list(self.player_features[player_id]), axis=0)

    def find_best_reid_match(self, features):
        """Find best re-identification match among disappeared players"""
        best_match_id = None
        best_similarity = 0.0

        for player_id in self.disappeared_players:
            avg_features = self.get_average_features(player_id)
            if avg_features is not None:
                similarity = self.calculate_feature_similarity(features, avg_features)
                if similarity > best_similarity and similarity > self.reid_threshold:
                    best_similarity = similarity
                    best_match_id = player_id

        return best_match_id, best_similarity

    def track_players(self, frame, frame_number):
        """Main tracking function"""
        detections = self.detect_players(frame)
        detection_features = [self.extract_player_features(frame, det) for det in detections]

        if len(self.active_players) == 0:
            for i, detection in enumerate(detections):
                if detection_features[i] is not None:
                    player_id = self.next_id
                    self.next_id += 1
                    self.active_players[player_id] = detection
                    self.update_player_features(player_id, detection_features[i])
                    self.tracking_history[player_id].append({
                        'frame': frame_number,
                        'bbox': detection,
                        'features': detection_features[i]
                    })
            return self.active_players

        active_player_ids = list(self.active_players.keys())
        cost_matrix = np.full((len(active_player_ids), len(detections)), 1000.0)

        for i, player_id in enumerate(active_player_ids):
            player_features = self.get_average_features(player_id)
            player_bbox = self.active_players[player_id]

            for j, detection in enumerate(detections):
                if detection_features[j] is not None:
                    distance = self.calculate_distance(player_bbox, detection)
                    distance_cost = distance / self.max_distance
                    similarity = self.calculate_feature_similarity(player_features, detection_features[j])
                    feature_cost = 1.0 - similarity
                    if distance < self.max_distance:
                        cost_matrix[i, j] = 0.7 * distance_cost + 0.3 * feature_cost

        if cost_matrix.size > 0:
            row_indices, col_indices = linear_sum_assignment(cost_matrix)
            new_active_players = {}
            used_detections = set()

            for row, col in zip(row_indices, col_indices):
                if cost_matrix[row, col] < 0.5:
                    player_id = active_player_ids[row]
                    detection = detections[col]
                    new_active_players[player_id] = detection
                    used_detections.add(col)
                    if detection_features[col] is not None:
                        self.update_player_features(player_id, detection_features[col])
                        self.tracking_history[player_id].append({
                            'frame': frame_number,
                            'bbox': detection,
                            'features': detection_features[col]
                        })

            for i, player_id in enumerate(active_player_ids):
                if i not in row_indices or cost_matrix[i, col_indices[list(row_indices).index(i)]] >= 0.5:
                    self.disappeared_players[player_id] = 0

            for j, detection in enumerate(detections):
                if j not in used_detections and detection_features[j] is not None:
                    reid_match, similarity = self.find_best_reid_match(detection_features[j])
                    if reid_match is not None:
                        new_active_players[reid_match] = detection
                        del self.disappeared_players[reid_match]
                        self.update_player_features(reid_match, detection_features[j])
                        self.tracking_history[reid_match].append({
                            'frame': frame_number,
                            'bbox': detection,
                            'features': detection_features[j],
                            'reidentified': True
                        })
                    else:
                        player_id = self.next_id
                        self.next_id += 1
                        new_active_players[player_id] = detection
                        self.update_player_features(player_id, detection_features[j])
                        self.tracking_history[player_id].append({
                            'frame': frame_number,
                            'bbox': detection,
                            'features': detection_features[j]
                        })

            self.active_players = new_active_players

        to_remove = []
        for player_id in self.disappeared_players:
            self.disappeared_players[player_id] += 1
            if self.disappeared_players[player_id] > self.max_disappeared:
                to_remove.append(player_id)
        for player_id in to_remove:
            del self.disappeared_players[player_id]

        return self.active_players

    def process_video(self, video_path, output_path=None):
        """Process entire video with tracking and re-identification"""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")

        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        frame_number = 0
        print(f"Processing video: {total_frames} frames at {fps} FPS")

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            active_players = self.track_players(frame, frame_number)
            for player_id, bbox in active_players.items():
                x1, y1, x2, y2 = bbox[:4]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Player {player_id}', (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            if output_path:
                out.write(frame)

            frame_number += 1
            if frame_number % 100 == 0:
                print(f"Processed {frame_number}/{total_frames} frames ({frame_number/total_frames*100:.1f}%)")

        cap.release()
        if output_path:
            out.release()

        print(f"Video processing completed. Total unique players tracked: {self.next_id}")
        return self.tracking_history

    def save_results(self, output_dir):
        """Save tracking results and statistics"""
        os.makedirs(output_dir, exist_ok=True)

        tracking_data = {}
        for player_id, history in self.tracking_history.items():
            tracking_data[player_id] = []
            for entry in history:
                entry_copy = entry.copy()
                if 'features' in entry_copy:
                    entry_copy['features'] = entry_copy['features'].tolist()
                tracking_data[player_id].append(entry_copy)

        with open(os.path.join(output_dir, 'tracking_summary.json'), 'w') as f:
            json.dump(tracking_data, f, indent=2)

        with open(os.path.join(output_dir, 'player_features.pkl'), 'wb') as f:
            pickle.dump(dict(self.player_features), f)

        stats = self.generate_statistics()
        with open(os.path.join(output_dir, 'tracking_statistics.json'), 'w') as f:
            json.dump(stats, f, indent=2)

        print(f"Results saved to {output_dir}")

    def generate_statistics(self):
        """Generate tracking statistics"""
        stats = {
            'total_players': self.next_id,
            'player_statistics': {}
        }

        for player_id, history in self.tracking_history.items():
            player_stats = {
                'total_detections': len(history),
                'first_frame': history[0]['frame'] if history else 0,
                'last_frame': history[-1]['frame'] if history else 0,
                'reidentification_events': sum(1 for entry in history if entry.get('reidentified', False))
            }
            player_stats['tracking_duration'] = player_stats['last_frame'] - player_stats['first_frame'] if len(history) > 1 else 0
            stats['player_statistics'][player_id] = player_stats

        return stats

    def visualize_tracking_paths(self, video_path, output_path, max_frames=300):
        """Create visualization of player tracking paths"""
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        cap.release()

        if not ret:
            return

        canvas = frame.copy()
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
                 (255, 0, 255), (0, 255, 255), (128, 0, 128), (255, 165, 0)]

        for player_id, history in self.tracking_history.items():
            if len(history) < 2:
                continue
            color = colors[player_id % len(colors)]
            centers = []
            for entry in history[:max_frames]:
                bbox = entry['bbox']
                center = ((bbox[0] + bbox[2]) // 2, (bbox[1] + bbox[3]) // 2)
                centers.append(center)
            for i in range(len(centers) - 1):
                cv2.line(canvas, centers[i], centers[i + 1], color, 2)
            if centers:
                cv2.circle(canvas, centers[0], 5, color, -1)
                cv2.circle(canvas, centers[-1], 8, color, 3)
                cv2.putText(canvas, f'P{player_id}', (centers[0][0] + 10, centers[0][1] - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.imwrite(output_path, canvas)

def main():
    # Configuration
    MODEL_PATH = "best.pt"  # Relative path
    INPUT_VIDEO = "15sec_input_720p.mp4"
    OUTPUT_VIDEO = "tracked_output.mp4"
    OUTPUT_DIR = "tracking_results"

    tracker = PlayerTracker(MODEL_PATH, max_disappeared=30, max_distance=150)
    print("Starting player tracking and re-identification...")
    tracking_history = tracker.process_video(INPUT_VIDEO, OUTPUT_VIDEO)
    tracker.save_results(OUTPUT_DIR)
    tracker.visualize_tracking_paths(INPUT_VIDEO, os.path.join(OUTPUT_DIR, 'tracking_paths.jpg'))

    stats = tracker.generate_statistics()
    print(f"\nTracking Summary:")
    print(f"Total players tracked: {stats['total_players']}")
    for player_id, player_stats in stats['player_statistics'].items():
        print(f"Player {player_id}: {player_stats['total_detections']} detections, "
              f"{player_stats['tracking_duration']} frames duration, "
              f"{player_stats['reidentification_events']} re-ID events")
    print(f"\nOutput video saved as: {OUTPUT_VIDEO}")
    print(f"Detailed results saved in: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

Starting player tracking and re-identification...
Processing video: 375 frames at 25 FPS

0: 384x640 1 ball, 16 players, 2 referees, 2308.5ms
Speed: 4.5ms preprocess, 2308.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2613.8ms
Speed: 3.1ms preprocess, 2613.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2888.7ms
Speed: 3.0ms preprocess, 2888.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2286.3ms
Speed: 2.9ms preprocess, 2286.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2297.4ms
Speed: 3.9ms preprocess, 2297.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2314.7ms
Speed: 3.1ms preprocess, 2314.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384

TypeError: Object of type ndarray is not JSON serializable